バックトレースを使って、初期値を一つずつ確かめていく必要がある。初期値の候補については、`n1`を1桁からはじめる。`f`を桁数としたとき、`f(n1) + f(n2)` = `f(num) / 2`が最大値。もし`f(n1) + f(n2) > f(num) / 2`になったときは、`n1`を2桁にしてやり直す。

バックトレースを実装するときは関数を細かく切り分ける方法が有効。まずはサンプルがちょうど正解があるので、`f1`と`f2`を与えたら、それに合わせて最後までチェックしてくれる関数を実装する。（まあ継続でもいんだけど、継続は記述量の割にはそれに即した効果を得ていない感じがあるので、ならば関数を繋げていって差し戻すほうが、直感的にも実装的にも解りやすい気はする）

In [20]:
(require racket/list)

(define (check-additive n1 n2 num-list)
  (let* ([sum (+ n1 n2)]
         [sum-length (string-length (number->string sum))])
    (cond 
     [(null? num-list) #t]
     [(> sum-length (length num-list)) #f]
     [else (let 
             ([try-num (string->number (list->string (take num-list sum-length)))])
             (if (= sum try-num)
                 (check-additive n2 try-num (drop num-list sum-length)) #f))])))

(require rackunit)
(check-equal? (check-additive 1 1 (string->list "2358")) #t)
(check-equal? (check-additive 1 1 (string->list "2359")) #f)

(define (set-n2 n1 num-list)
  (define (inner n1 lst2 num-list)
    (let ([n2 (string->number (list->string lst2))])
      (cond [(> (min (string-length (number->string n1)) (length lst2)) (length num-list)) #f]
            [(and (> (length lst2) 1) (char=? (car lst2) #\0)) #f]
            [(check-additive n1 n2 num-list) #t]
            [else (inner n1 (append lst2 (list (car num-list))) (cdr num-list))])))
  (inner n1 (list (car num-list)) (cdr num-list)))

(define (set-n1 num-list)
  (define (inner lst1 num-list)
    (let ([n1 (string->number (list->string lst1))])
      (cond [(> (length lst1) (length num-list)) #f]
            [(and (> (length lst1) 1) (char=? (car lst1) #\0)) #f]
            [(set-n2 n1 num-list) #t]
            [else (inner (append lst1 (list (car num-list))) (cdr num-list))])))
  (inner (list (car num-list)) (cdr num-list)))

(check-equal? (set-n2 1 (string->list "99100199")) #t)
(check-equal? (set-n1 (string->list "199100199")) #t)
(check-equal? (set-n1 (string->list "112358")) #t)
(check-equal? (set-n1 (string->list "1023")) #f)

解けたのはいいけれど、野菜が生切れのようなコードになっているので本当に良くないと思う。解いたあとに気になった点としては、先頭にゼロがあるかどうかの判定関数はあるべきのように感じる。あとテストケースに対する「可能性」に対してあまり敏感ではなく、記述漏れを起こして違う結果が出ることが多いので、そこのところの「悪い意味での不注意さ」を直す必要がある。